In [ ]:
import numpy as np
import pandas as pd
import os 
import pickle
import keras 
from keras.datasets import cifar10, mnist
import numpy as np
import cv2
from matplotlib import pyplot as plt
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Conv1D, Conv3D, Input, Activation, Subtract, MaxPool2D, UpSampling2D
import tensorflow
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from math import log10, sqrt 
from keras.utils.np_utils import to_categorical  

Using TensorFlow backend.


In [ ]:
#import datasets needed

(train_images_raw,train_label),(test_images_raw,test_label)=cifar10.load_data()

In [ ]:
#convert labels into one-hot-encoded vectors
def label_vectors():
  train_labels=to_categorical(train_label,num_classes=10)
  test_labels=to_categorical(test_label, num_classes=10)
  return train_labels, test_labels


In [ ]:
#function to add noise
def add_gauss(img):
  noise_added_images=[]
  for i in range(len(img)):
    noise_added_images.append(img[i]+1.5*np.random.normal(0,0.1,(32,32,3)))
  return noise_added_images


In [ ]:
def statistical_filter(img):
  filtered_images=[]
  for i in range(len(img)):
    my_img=img[i]
    X=cv2.GaussianBlur(my_img,(5,5),cv2.BORDER_DEFAULT)
    filtered_images.append(X)
  return filtered_images

In [ ]:
def cnn_denoiser(train_images_noisy,train_images):
  dilation=[1,1,1,1,1,1,1]
  filter_size=[32,32,64,64,128,64,64]
  input_image= Input(shape=(28,28,1))
  x= Conv2D(64,kernel_size=3,activation="relu",dilation_rate=1,strides=(1,1),padding="same")(input_image)
  
  for layers in range(7):
    x=Conv2D(64,kernel_size=3, strides=(1,1),dilation_rate=1,padding="same")(x)
    x= BatchNormalization(axis=-1,epsilon=0.001 )(x)
    x=Activation("relu")(x)

  x=Conv2D(1, kernel_size=3,strides=(1,1),dilation_rate=1,padding="same")(x)
  x=Subtract()([input_image,x])
  model=Model(inputs=input_image, outputs=x)
  #model.compile(optimizer="Adam", loss=["mean_squared_logarithmic_error"])
  model.compile(optimizer="Adam", loss=["mse"])
  
  A=np.array(train_images_noisy)
  B=np.array(train_images)
  model.fit(A,B,epochs=2)
  return model


In [ ]:
def encoder(input_img):
    """
    Encoder function.
    Arguments:
      input_img: Input image, Tensor of shape (28, 28, 1)
    
    Returns: Encoded image, Tensor of shape (7, 7, 8)
    """
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='encoder_conv1')(input_img)
    x = MaxPool2D((2, 2), padding='same', name='encoder_pool1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='encoder_conv2')(x)
    x = MaxPool2D((2, 2), padding='same', name='encoder_pool2')(x)
    encoded = Conv2D(8, (3, 3), activation='relu', padding='same', name='encoder_conv3')(x)
    return encoded


def decoder(encoded):
    """
    Decoder function.
    Arguments:
      encoded: Encoded image, Tensor of shape (7, 7, 8)
    
    Returns: Decoded images, Tensor of shape (28, 28, 1)
    """
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='decoder_conv1')(encoded)
    x = UpSampling2D((2, 2), name='decoder_upsample1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='decoder_conv2')(x)
    x = UpSampling2D((2, 2), name='decoder_upsample2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='decoder_conv3')(x)
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same', name='decoder_conv4')(x)
    return decoded

In [ ]:
def autoencoder_denoiser(x_train_noisy,x_train_expanded,x_test_noisy,x_test_expanded):
  input_img = Input(shape=(32, 32, 3))

  #create autoencoder model
  autoencoder = Model(input_img, decoder(encoder(input_img)))
  #complile model
  autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
  #define callback for earlystopping to stop training if the validation loss does not decrese for 10 epochs 
  earlyStopping = EarlyStopping(monitor='val_loss', patience=2)

  #define callback to save model weights if the validation loss improves
  modelChkpt = ModelCheckpoint(filepath='best_weights.h5', monitor='val_loss', save_best_only=True) 

  #train model
  train_history = autoencoder.fit(x_train_noisy, x_train_expanded,
                epochs=100,
                batch_size=128,
                shuffle=True,
                validation_data=(x_test_noisy, x_test_expanded),callbacks=[modelChkpt, earlyStopping])
  return autoencoder
  

In [ ]:
##classification model for noisy images
def classification(train_images,train_labels,test_images,test_labels):
  early_stop=[EarlyStopping(patience=2)]
  model=Sequential()
  model.add(Conv2D(32,kernel_size=3,activation='relu', input_shape=(32,32,3)))
  model.add(Conv2D(128, kernel_size=3,activation='relu'))
  model.add(Flatten())
  model.add(Dense(10,activation='softmax'))

  model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

  train_images=np.array(train_images)
  test_images=np.array(test_images)
  
  model.fit(train_images,train_labels,validation_data=(test_images,test_labels),epochs=10)
  return model

In [ ]:
test_images = test_images_raw.astype('float32') / 255.0
train_images= train_images_raw.astype('float32') / 255.0



In [ ]:
train_images = np.expand_dims(train_images_raw, axis=3)
test_images = np.expand_dims(test_images_raw, axis=3)  


In [ ]:
noisy_test_images=add_gauss(test_images)

In [ ]:
noisy_train_images=add_gauss(train_images)


In [ ]:
train_labels, test_labesl=label_vectors()


In [ ]:
filtered_images_train=statistical_filter(noisy_train_images)
filtered_images_test=statistical_filter(noisy_test_images)
autoencoder=autoencoder_denoiser(np.array(filtered_images_train),np.array(train_images), np.array(filtered_images_test),np.array(test_images))


In [ ]:
print(np.array(noisy_train_images).shape)
train_images_expanded = np.expand_dims(train_images, axis=3)
  test_images_expanded = np.expand_dims(test_images, axis=3) 

In [ ]:
model_noisy=classification(noisy_train_images,train_labels,noisy_test_images,test_labesl)


In [ ]:
filtered_images_train=np.expand_dims(filtered_images_train,axis=3)
filtered_images_test=np.expand_dims(filtered_images_test,axis=3)
model_denoised=classification(filtered_images_train,train_labels,filtered_images_test,test_labesl)


In [ ]:
import tensorflow as tf
im1 = tf.image.convert_image_dtype(noisy_test_images, tf.float32)
im2 = tf.image.convert_image_dtype(filtered_images_test, tf.float32)
psnr = tf.image.psnr(im1, im2, max_val=1.0)
psnr_train = np.mean(psnr)
print(psnr_train)